# Парсинг HTML страниц.
# Поиск и извлечение ссылки на следующую страницу

**Соберем обложки, цены и названия с сайта по продаже книг https://books.toscrape.com**

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
# получаем код страницы и статус код
get_html = requests.get('https://books.toscrape.com/')
get_html.status_code

Статус код 200 означает, что запрос прошел без проблем и мы подключились к сайту

# Парсинг HTML страниц.

In [ ]:
# Получим html
html = get_html.content
html

In [ ]:
# спарсим страницу 
soup = bs(html, 'html.parser')
soup

Включаем исследование страницы в браузере. Выбираем <section>, где хранится весь раздел с книгами. Ищем нужные теги div со списком контента.

In [ ]:
sections = soup.select('section')
sections

In [ ]:
len(sections) # У нас 1 секция с тегами. Их может быть больше.

In [ ]:
section = sections[0]

select_one выбирает первый попавшийся тег. Если их много (у нас сейчас он 1), то нужно указывать select.

In [ ]:
# Получим блок ко списком книг
books_block = section.select_one('ol[class=row]')
books_block

In [ ]:
# Получим все книги. Их 20 на транице
books = books_block.select('li')
print(len(books))

**Получим обложки, заголовки и цены.**
Чтобы узнать полный путь, кликаем правой кнопкой мыши на обложку и открываем в новой вкладке и копируем часть адреса из строки.


In [ ]:
books_data = []
for book in books:
    image = 'https://books.toscrape.com/' + book.find('div', attrs = {'class': 'image_container'}).find('img')['src']
    title = book.find('h3').find('a')[title]
    price = book.find('p', attrs = {'class': 'price_color'}).text
    
    book_dict = {
        'image': image,
        'title': title,
        'price': price
    }
    books_data.append(book_dict)

print(len(books_data))

In [ ]:
books_data[7]

# Поиск и извлечение ссылки на следующую страницу

In [ ]:
get_html = requests.get('https://books.toscrape.com/catalogue/page-1.html')
if get_html.status_code == 200:
    soup = bs(get_html.content, "html.parser")
    next_page = 'https://books.toscrape.com/catalogue/' + soup.find('li', attrs={'class': 'next'}).find('a')['href']

    get_next_html = requests.get(next_page)
    if get_next_html.status_code == 200:
        print(get_next_html.content)

# Автоматизируем процесс

In [ ]:
def get_books(content):
    soup = bs(content, 'html.parser')
    ol = soup.find('ol', attrs={'class': 'row'})
    books = ol.select('li')

    books_data = []
    for book in books:
        image = 'https://books.toscrape.com/' + book.find('div', attrs = {'class': 'image_container'}).find('img')['src']
        title = book.find('h3').find('a')[title]
        price = book.find('p', attrs = {'class': 'price_color'}).text
    
        book_dict = {
            'image': image,
            'title': title,
            'price': price
        }
        books_data.append(book_dict)
    return books_data

In [ ]:
def get_next_page(content):
    soup = bs(content, html.parser')
    try:
        next_page = 'https://books.toscrape.com/catalogue/' + soup.find('li', attrs={'class': 'next'}).find('a')['href']
        return next_page
    concept:
        pass

In [ ]:
final_data = []
page_number = 1
url = 'https://books.toscrape.com/catalogue/page-1.html'
get_html = requests.get(url)
if get_html.status_code == 200:
    while True:
        books = get_books(get_html.content)
        print(f'Получено {len(BOOKS)} с {page_number} страницы')
        final_data += books

        next_page = get_next_page(get_html.content)
        if next_page:
            page_number += 1
            get_html = requests.get(next_page)
            if get_html.status_code == 200:
                print(f'Начинаем парсить {page_number}э)
        else:
            break
print(f"Данные спарсились: {page_number} страниц, {len(final_data}) книг")